# KOPIS 공연추천 시스템 테스트

KOPIS OpenAPI를 활용하여 공연 정보를 수집하고, OCR과 FastText를 이용한 추천 시스템을 구현합니다.

## 필요한 라이브러리 설치

## 라이브러리 임포트

In [21]:
import cv2
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance
import pytesseract
from io import BytesIO
from gensim.models import FastText as FastText
import re
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv
import os

## KOPIS API 클라이언트 클래스 정의

In [22]:
# .env 파일 로드
load_dotenv()

# API 키를 .env 파일에서 가져오기
KOPIS_API_KEY = os.getenv('KOPIS_API_KEY')

In [23]:
class KopisAPI:
    def __init__(self, service_key):
        self.service_key = service_key
        self.base_url = "http://www.kopis.or.kr/openApi/restful"
    
    def get_performance_list(self, start_date, end_date):
        """공연 목록 조회"""
        url = f"{self.base_url}/pblprfr"
        params = {
            'service': self.service_key,
            'stdate': start_date,
            'eddate': end_date,
            'rows': 100,
            'cpage': 1,
            'shcate': 'AAAA|GGGA'  # 연극(AAAA)과 뮤지컬(GGGA)만 조회
        }
        response = requests.get(url, params=params)
        root = ET.fromstring(response.content)
        
        performances = []
        for db in root.findall('.//db'):
            perf = {}
            for child in db:
                perf[child.tag] = child.text
            performances.append(perf)
        
        return performances
    
    def get_performance_detail(self, mt20id: str) -> Optional[Dict[str, Any]]:
        """공연 상세정보 조회 - 포스터와 소개이미지 모두 처리"""
        url = f"{self.base_url}/pblprfr/{mt20id}"
        params = {'service': self.service_key}
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            root = ET.fromstring(response.content)
            db = root.find('.//db')
            
            if db is None:
                return None
                
            detail = {}
            for elem in db:
                if elem.tag == 'styurls':
                    # XML 구조 디버깅
                    print(f"styurls element found for {mt20id}")
                    print(f"styurls content: {ET.tostring(elem, encoding='unicode')}")
                    
                    # 소개이미지 목록 추출 (수정된 XPath)
                    urls = []
                    for styurl in elem.findall('styurl'):
                        if styurl.text and styurl.text.strip():
                            print(f"Found image URL: {styurl.text}")
                            urls.append(styurl.text.strip())
                    detail['styurls'] = urls
                else:
                    if elem.text and elem.text.strip():
                        detail[elem.tag] = elem.text.strip()
                    
            # 디버깅을 위한 출력
            if 'styurls' in detail:
                print(f"Total styurls found for {mt20id}: {len(detail['styurls'])}")
            else:
                print(f"No styurls found for {mt20id}")
                
            return detail
            
        except Exception as e:
            print(f"API 요청 오류: {e}")
            return None

## 텍스트 처리 클래스 정의

In [24]:
class TextProcessor:
    def __init__(self):
        self.model = None
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    def enhance_image(self, img):
        # [기존 코드 유지]
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.0)
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(2.0)
        return img
    
    def preprocess_image(self, img_array):
        # [기존 코드 유지]
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)
        _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        processed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        return processed
    
    def get_image_sections(self, img):
        # [기존 코드 유지]
        width, height = img.size
        sections = []
        section_height = height // 3
        for i in range(3):
            top = i * section_height
            bottom = (i + 1) * section_height
            section = img.crop((0, top, width, bottom))
            sections.append(section)
        return sections

    def extract_plot_only(self, text):
        """공연 내용 관련 텍스트만 추출하는 함수"""
        # 제외할 키워드들
        exclude_keywords = [
            '관람안내', '관람 안내', '공연관람', '공연 관람', 
            '예매', '티켓', '입장', '객석', '수령', 
            '환불', '취소', '주의사항', '공지사항', '공연시간',
            'creative team', '제작', '작가', '연출', '작곡',
            '문의', '안내', '관람료', '주차', '가격',
            '예매처', '문자', '전화', '공연장소', '찾아오시는',
            '기획', '후원', '협찬', '주최', '주관'
        ]
        
        def is_info_sentence(sentence):
            return any(keyword in sentence for keyword in exclude_keywords)
        
        def has_meaningful_content(sentence):
            korean_chars = len(re.findall(r'[가-힣]', sentence))
            special_chars = len(re.findall(r'[^가-힣a-zA-Z\s\.,!?()]', sentence))
            total_chars = len(sentence)
            
            if total_chars == 0:
                return False
            
            special_ratio = special_chars / total_chars
            min_korean_chars = 10
            return korean_chars >= min_korean_chars and special_ratio < 0.3 and total_chars >= 15

        # 문장 단위로 분리
        sentences = re.split(r'[.!?\n]\s*', text)
        plot_sentences = []
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not is_info_sentence(sentence) and has_meaningful_content(sentence):
                cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s\.,!?()]', ' ', sentence)
                cleaned = re.sub(r'\s+', ' ', cleaned).strip()
                if cleaned:
                    plot_sentences.append(cleaned)
        
        if plot_sentences:
            # 중복 제거 및 정리
            seen_sentences = set()
            unique_sentences = []
            for sentence in plot_sentences:
                sentence_start = ' '.join(sentence.split()[:5])
                if sentence_start not in seen_sentences:
                    seen_sentences.add(sentence_start)
                    unique_sentences.append(sentence)
            
            # 문장들을 결합하고 추가 정제
            text = ' '.join(unique_sentences)
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'\b(?!\d{4}\b)\d+\b', '', text)
            text = re.sub(r'\b[a-zA-Z]\b', '', text)
            text = re.sub(r'[^\w\s가-힣]', ' ', text)
            
            return text.strip()
        
        return ""
    
    def extract_text_from_image(self, image_url):
        # [기존 코드 유지]
        try:
            print(f"이미지 다운로드 시도: {image_url}")
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            
            if img.format == 'GIF':
                img = img.convert('RGB')
            
            target_width = 1000
            width_percent = (target_width / float(img.size[0]))
            target_height = int(float(img.size[1]) * float(width_percent))
            img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
            
            enhanced_img = self.enhance_image(img)
            img_array = np.array(enhanced_img)
            processed_img = self.preprocess_image(img_array)
            sections = self.get_image_sections(img)
            
            texts = []
            for i, section in enumerate(sections):
                configs = [
                    '--oem 3 --psm 6',
                    '--oem 3 --psm 1',
                    '--oem 3 --psm 4'
                ]
                
                section_texts = []
                for config in configs:
                    text = pytesseract.image_to_string(
                        section, 
                        lang='kor+eng',
                        config=config
                    )
                    if text.strip():
                        section_texts.append(text)
                
                if section_texts:
                    longest_text = max(section_texts, key=len)
                    texts.append(longest_text)
            
            processed_text = pytesseract.image_to_string(
                processed_img,
                lang='kor+eng',
                config='--oem 3 --psm 6'
            )
            texts.append(processed_text)
            
            combined_text = ' '.join(texts)
            cleaned_text = self.clean_text(combined_text)
            
            print(f"추출된 총 텍스트 길이: {len(cleaned_text)}")
            print(f"텍스트 샘플: {cleaned_text[:200]}...")
            
            return cleaned_text
            
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {str(e)}")
            return ""

    def extract_meaningful_text(self, text):
        # [기존 코드 유지]
        def is_meaningful_sentence(sentence):
            korean_chars = len(re.findall(r'[가-힣]', sentence))
            min_korean_chars = 10
            special_chars = len(re.findall(r'[^가-힣a-zA-Z\s\.,!?()]', sentence))
            total_chars = len(sentence)
            
            if total_chars == 0:
                return False
                
            special_ratio = special_chars / total_chars
            return korean_chars >= min_korean_chars and special_ratio < 0.3

        sentences = re.split(r'[.!?]\s+', text)
        meaningful_sentences = []
        
        for sentence in sentences:
            sentence = sentence.strip()
            if is_meaningful_sentence(sentence):
                cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s\.,!?()]', ' ', sentence)
                cleaned = re.sub(r'\s+', ' ', cleaned).strip()
                if cleaned:
                    meaningful_sentences.append(cleaned)
        
        if meaningful_sentences:
            text = ' '.join(meaningful_sentences)
            words = text.split()
            unique_words = []
            for word in words:
                if word not in unique_words[-3:]:
                    unique_words.append(word)
            text = ' '.join(unique_words)
            text = re.sub(r'\b\d+\b', '', text)
            text = re.sub(r'\b[a-zA-Z]\b', '', text)
            return text.strip()
        
        return ""
    
    def clean_text(self, text):
        """텍스트 전처리"""
        if not text:
            return ""
        
        # 공연 내용만 추출
        plot = self.extract_plot_only(text)
        if plot:
            return plot.lower()
        
        # 기존 텍스트 정제 방식을 백업으로 사용
        meaningful_text = self.extract_meaningful_text(text)
        if meaningful_text:
            return meaningful_text.lower()
        
        # 가장 기본적인 전처리
        text = re.sub(r'[^\w\s가-힣]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = text.replace('\n', ' ')
        words = text.split()
        words = list(dict.fromkeys(words))
        text = ' '.join(words)
        
        return text.strip().lower()

    def train_model(self, texts):
        # [기존 코드 유지]
        texts = [text for text in texts if text.strip()]
        if not texts:
            print("경고: 학습할 텍스트가 없습니다.")
            return
            
        sentences = [[word for word in text.split()] for text in texts]
        try:
            self.model = FastText(
                sentences=sentences, 
                vector_size=100, 
                window=5, 
                min_count=1,
                workers=4
            )
            print(f"모델 학습 완료: {len(sentences)} 문장")
        except Exception as e:
            print(f"모델 학습 오류: {str(e)}")
    
    def get_text_vector(self, text):
        # [기존 코드 유지]
        if self.model is None:
            print("경고: 모델이 학습되지 않았습니다.")
            return np.zeros(100)
            
        words = text.split()
        word_vectors = [self.model.wv[word] for word in words if word in self.model.wv]
        if not word_vectors:
            return np.zeros(100)
        return np.mean(word_vectors, axis=0)

## 공연 추천 시스템 클래스 정의

In [25]:
class PerformanceRecommender:
    def __init__(self, api_client, text_processor):
        self.api_client = api_client
        self.text_processor = text_processor
        self.performances_df = None
    
    def collect_performance_data(self, days=30):
        """공연 데이터 수집 - 모든 이미지 처리"""
        start_date = datetime.now().strftime("%Y%m%d")
        end_date = (datetime.now() + timedelta(days=days)).strftime("%Y%m%d")
        
        performances = []
        perf_list = self.api_client.get_performance_list(start_date, end_date)
        
        for perf in perf_list[:10]:  # 테스트를 위해 10개만
            mt20id = perf['mt20id']
            detail = self.api_client.get_performance_detail(mt20id)
            
            if detail:
                # 포스터 텍스트 추출
                poster_text = ""
                if 'poster' in detail and detail['poster'] and detail['poster'].startswith('http'):
                    try:
                        poster_text = self.text_processor.extract_text_from_image(detail['poster'])
                    except Exception as e:
                        print(f"포스터 이미지 처리 오류({mt20id}): {str(e)}")
                
                # 소개이미지 텍스트 추출
                intro_texts = []
                if 'styurls' in detail and isinstance(detail['styurls'], list):
                    for img_url in detail['styurls']:
                        if img_url and img_url.startswith('http'):
                            try:
                                text = self.text_processor.extract_text_from_image(img_url)
                                if text:
                                    intro_texts.append(text)
                            except Exception as e:
                                print(f"소개이미지 처리 오류({mt20id}): {str(e)}")
                
                # 모든 텍스트 결합
                all_text = ' '.join(filter(None, [poster_text] + intro_texts))
                
                performances.append({
                    'mt20id': mt20id,
                    'title': detail.get('prfnm', ''),
                    'plot': all_text if all_text.strip() else ""
                })
        
        self.performances_df = pd.DataFrame(performances)
        return self.performances_df
    
    def prepare_model(self):
        """추천 모델 준비"""
        if self.performances_df is None:
            raise ValueError("공연 데이터를 먼저 수집하세요.")
        
        plots = self.performances_df['plot'].tolist()
        self.text_processor.train_model(plots)
    
    def get_recommendations(self, user_plot, top_n=5):
        """사용자 입력에 기반한 공연 추천"""
        if self.performances_df is None:
            raise ValueError("공연 데이터를 먼저 수집하세요.")
        
        user_vector = self.text_processor.get_text_vector(user_plot)
        
        # 각 공연과의 유사도 계산
        similarities = []
        for plot in self.performances_df['plot']:
            plot_vector = self.text_processor.get_text_vector(plot)
            similarity = np.dot(user_vector, plot_vector) / (
                np.linalg.norm(user_vector) * np.linalg.norm(plot_vector)
            )
            similarities.append(similarity)
        
        self.performances_df['similarity'] = similarities
        recommendations = self.performances_df.nlargest(top_n, 'similarity')
        return recommendations[['title', 'similarity']]

## 시스템 테스트

아래 셀에서 실제 테스트를 수행합니다. API 키를 설정하고 실행해보세요.

In [26]:
# 테스트 코드
if __name__ == "__main__":
    # API 키 설정
    SERVICE_KEY = KOPIS_API_KEY
    
    # 시스템 초기화
    api_client = KopisAPI(SERVICE_KEY)
    text_processor = TextProcessor()
    recommender = PerformanceRecommender(api_client, text_processor)
    
    # 데이터 수집
    print("데이터 수집 중...")
    performances_df = recommender.collect_performance_data()
    print("\n수집된 공연 데이터:")
    print(performances_df.head())

데이터 수집 중...
styurls element found for PF257033
styurls content: <styurls>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF257033_250110_0148450.jpg</styurl>
        </styurls>
        
Found image URL: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF257033_250110_0148450.jpg
Total styurls found for PF257033: 1
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmPoster/PF_PF257033_250110_134845.gif
추출된 총 텍스트 길이: 47
텍스트 샘플: ㆍ 4 7 사 6 음 on 늘 v e q elo 날 apts 기억게 나는 너른 i a...
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF257033_250110_0148450.jpg
추출된 총 텍스트 길이: 212
텍스트 샘플: 뮤지컬 무명  준희 를 처음으로 만나는 뮤지컬 무명 준희 속 인물들의첫인사부터주요 넘버시연은 be  연습과정중에있었던여러 고민들과 소중한 기억들을 꺼내보는 비밀모임 세명의 인물들이소개하는 각자의 af 무명  준희 를 만들어가며 li 고민과 어려움 무명 준희 와 sty 오래도록 기억하고 싶은 찰나 그리고찰나를포착하는시간 조선어학회 비밀모임의 소중한 기억들 은...
styurls element found for PF257012
styurls content: <styurls>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF257012_250110_1250530.jpg</styurl>
        </styurls>
    

In [27]:
pd.set_option('display.max_colwidth', None)

In [28]:
performances_df

,mt20id,title,plot
0,PF257033,"무명, 준희: 작은음악회",ㆍ 4 7 사 6 음 on 늘 v e q elo 날 apts 기억게 나는 너른 i a 뮤지컬 무명 준희 를 처음으로 만나는 뮤지컬 무명 준희 속 인물들의첫인사부터주요 넘버시연은 be 연습과정중에있었던여러 고민들과 소중한 기억들을 꺼내보는 비밀모임 세명의 인물들이소개하는 각자의 af 무명 준희 를 만들어가며 li 고민과 어려움 무명 준희 와 sty 오래도록 기억하고 싶은 찰나 그리고찰나를포착하는시간 조선어학회 비밀모임의 소중한 기억들 은 박선영 홍성원 강병훈
1,PF257012,하숙집 [부산],출연 오혜정 김명수 김호현 이보람 전수진 깅 나네 신서영 김지훈 토 일 토 4시 7시 일 5시 일터소극징 출연 오혜경 김명수 김호현 oe 건수진 강연우 걸다현 바창호 ha oe 토 일 트 4시 7시 일유치 salt 일떠소극장 출연 오혜정 김명수 김호현 이보람 전수진 깅 나네 신서영 김지훈 토 일 토 4시 7시 일 5시 일터소극징 출연 오혜경 김명수 김호현 oe 건수진 강연우 걸다현 바창호 ha oe 토 일 트 4시 7시 일유치 salt 일떠소극장
2,PF256986,낭독x시대 1탄: 보도지침 X 초선의원,heri 토 임찬민 송영미 gore ont 전하영 정단비최이레 일 3시 김태곤 fsm es wes s801 oles aay 정단비 최이리 보오지소 1c rarer ron 어사계 전하연 전단비 arey 토 얼찬턴 송영미 pe 이사계 전허영 정단비 최이래 일 시 ud 유회제 war se 송영미 oas bd 정단비 최이래 일 7시 sea orm 혀영손 김건호 김늘매 김기주 김예별 heri 토 임찬민 송영미 gore ont 전하영 정단비최이레 일 3시 김태곤 fsm es wes s801 oles aay 정단비 최이리 보오지소 1c rarer ron 어사계 전하연 전단비 arey 토 얼찬턴 송영미 pe 이사계 전허영 정단비 최이래 일 시 ud 유회제 war se 송영미 oas bd 정단비 최이래 일 7시 sea orm 혀영손 김건호 김늘매 김기주 김예별
3,PF256985,옥뱅이뎐 [인천],어긋나는 데서 시작되는 이야기 노 예술인패스 할인 10h 예술인패스 카드소지자 본인 초 중 고 대학생 및 군인 대상 학생 군인 할인 1인1매 증빙 가능 서류 학생증 청소년증 신분증 여권 군인휴가증 공무원증 등 모든 할인은 실관람자 기준으로 할인혜택은 한 가지만 적용 가능합니다 정확한 할인 대상자 확인을 위해 증빙자료 외 개인정보를 추가 확인할 수 있습니다 어긋나는 데서 시작되는 우리의 이야기 소리꾼 이자현 dpe 영감찾는 ch che 상좌중 청명 st brian park 영감 성기옹 배뱅이는 ais 떠난 상좌중이 ss 하느라 오지 않는지 알고 죽어야겠다며 필사적으로 죽음을 거부한다 배뱅이가 죽지 않으면 이야기를 끝낼 수 없는 이자현은 os 수 없이 배뱅이와 함께 sass 만나기 위한 hss 떠난다 eal 하지만 oferi 배뱅이굿 에 등장하는 에 인돌이 아닌 사탐즐이 나타나기 시삭아는네 이자현은 어떻게든 원작대로 배뱅이가 죽기만 하면 이야기의 흐름을 되돌릴 수 있다고 생각한다 어떻게든 배뱅이굿 을 완창 하고 싶은 소리꾼 이자현과 인생의 at 맛을 알아가며 삶의 의미를 깨닫는 배뱅이 꼬여버린 공연과 그들의 운명은 소리꾼 otol 서도소리의 꾸 퍼렴미쿠 배뱅어는 자신을 gbe 강파종이 s44 하느라 도지 va tl chs 수 sho 배백이와 lo eee 만나기 sie 여정 떠난다 하지만 어편지 mos 통장하는 obo 아닌 사달들이 나타나기 시작하는데 이파설근 어떨게는 닌팍대로 배밴이가 조기만 하면 ook sess 되든 인 수 웠다고 생곽찬다 머범게든 배뱅이굿 완창 하고 as 소리꾼 이자련과 김승현 방미나 mao 서영민 이정민 최민우
4,PF256961,미녀와 야수 [광주],공연군의 gy obo 5199 5257 betel 쿨페이스 oat 2s 채닝 행혹을 주는 기륙극장 공연군의 gy obo 5199 5257 betel 쿨페이스 oat 2s 채닝 행혹을 주는 기륙극장 옛날 어느 먼 나라에 사는 한 왕자의 성에 한 누추한 차림의 노피가 찾아와 소 르바 므기 시즈 거으 hain 다레근 ato st s012 거네다 그러나 왕자가 장미와 노파의 초라함을 비웃자 노파는 마범을 걸어 그를 이수의 9402 바꾼 뒤 성을 wd 안에 사는 모든 것에 강력한 마번을 건다 이 그리고 진실한 사링을 얻지 못하면 세월이 흘러 인근 마을에 사는 bee geist 발명품 cusion 나가기 위해 숲속을 지나다 에 늦대에게 쫓겨 us 잃고 야수가 사는 미범의 성으로 옛날 어느 먼 나라에 사는 한 왕자의 성에 개 한 누추한 차림의 노파가 찾아와 전 하롯밤묵게 해줄 것을 stoi gus 장미 한 송이를 끈년다 그러나 왕자7 장미와 노파의 초라함을 비웃자 세 노파는 마법을 걸어 그를 야수의 모습으로 바꾼 뒤 ve 성을 포항한 그 안에 사는 모든 것에 강력한 마법을 건다 별 영원히 야수 모습으로 살 것이라는 말을 낭기고 사라진다 ap 세월이 옴러 인근 마을에 사는 ye spie 발명품 매회에 ut 위해 숲속을지나다 wed 독 ee 녹대에게 쫓겨 길을 잃고 이수가 사는 마범의 yor uses 캐릭터들의 잉스미은 동작들
5,PF256851,인생 내 컷,geet 공인 창작고안 1우드 정기공인 의 세제는 눈무산 빚으로 가득 재워졌이 신 박특앙 박수민 ce carre ons one orm at 장연주 정 가장 기억하고 싶은 사랑 개 가장 기억교싶은 순간 flak 그린 세상에 있은 fale or bast geue 알았다 당신의 앙에 gn ee 오울을 선울길게 들 hy be os he 그 순안 그러고 1g 4b 일어설 때 뼈아 gorter 상관없에 우원 같은 시간을 loratelnel 가장 기억라고 싶은 사람 자 es 뜨시고 ofl 셋 하면 찍습니다 우허가 그린 세상에 있은 fale of bass 슬근지오 삽았다 그저 gol ote yeg 잡고 있은 여가 보였다
6,PF256847,마음,() 예술공간 me ce 에조 주배짜지시제개 ag ft 그 고여 a4 다 자 펜을 내려놓을까 수없이 고민했지만 반드시 전해야할 이야기가 있어 글을 이어간다 그렇게 남자의 이야기가 시작된다 부모님을 일찍 여원 sal 숙부의 손에 자라며 그를 아버지처럼 따랐다 그러나 속부는 도 속이고 아버지의 재산을 빠돌리고 있었다 이 ams 알게 된 eal vhs 떠나 새로운 삶을 찾아 saez 향한다 도쿄의 한 하숙집에 정착한 도쿠는 다정한 하숙집 아주머니와 아가씨의 배려 속에 조금씩 마음의 상처를 치유해 나간다 그리고 자신처럼 아픈 과거를 지닌 친구 를 하숙 의상 소품 김다연 이경원 출연진 유독현 장미예 박구응 조성현 김다연 이경원 사람은 참 어리석고 간사하다 한 선배의 이 말이 오래도록 기억에 남습니다 간사함 은 사람이 원칙을 따르지 않고 자신의 이익에 따라 변하는 성질이라 생각합니다 한결같아보이는 사람도 상황에 따라 변할 수 밖에 없지요 많은 이들이 환경과 이익에 따라 끊임없이 변하고 심지어 가까웠던 사이도 이익 앞에서는 쉽게 돌아섭니다 인간이기에 어리석은 간사함에 안타까움을 느끼면서도 haa 익숙하고 공감이 되어 누구를 탓하기 어려운 마음입니다 저희가 각색한 연극 마음 은 이러한 인간의 마음을 조명합니다 사소한 계기로 마음이 변하고 스스로 파멸에 이르는 모습을 보여줍니다 세상에 악인은 존재하지 않는다 다만 누구나 악인이 될 수 있다 우리도 모르는 사이 누군가에게 악인이 된 적은 없었을까요 도쿠 조성현 남자 유독현 시즈 나나코 이경원 시즈 나나코 김다연 박구용 아주머니 장미예 시즈 ga 이경원 이경원 ade 이경원 박구용 박구용 박구용 박구용 박구용 아주머니 장미예 장미예 장미예 장미예 장미예 나나코 이경원 ade 김다연 이경원 ade 예술인 할인 000원 본인만 해당 예술인패스 제시 시 청소년 할인 0008 본인만 해당 24세 이하 신분증 제시 시 마음 할인 000원 소설 마음 지참 시 종로구민 할인 000원 본인만 해당 신분증 제시 시 장애인 할인 0008 복지카드 제시 시 동반 1인까지 복지 할인 000원 본인만 해당 국가유공자증 및 문화누리 카드 제시

In [29]:
# 모델 학습
print("모델 학습 중...")
recommender.prepare_model()

모델 학습 중...
모델 학습 완료: 10 문장


In [30]:
# 추천 테스트
test_input = "조선시대와 현대의 만남"
print(f"테스트 입력: {test_input}")

recommendations = recommender.get_recommendations(test_input)
print("\n추천 결과:")
display(recommendations)

테스트 입력: 조선시대와 현대의 만남

추천 결과:


,title,similarity
5,인생 내 컷,0.219993
4,미녀와 야수 [광주],0.176649
0,"무명, 준희: 작은음악회",0.097279
9,언제는 행복하지 않은 순간이 있었나요 (언행순) [서울],0.088935
7,엘사의 생일파티 제2탄: 얼음공주 이야기 [광주],0.060090


## 결과 분석

1. 추천된 공연들의 유사도 점수 분포
2. OCR 텍스트 추출 품질
3. FastText 모델의 성능